In [6]:
import tensorflow as tf
import numpy as np

#set hyperparameters
max_len = 40
step = 2
num_units = 128
learning_rate = 0.001
batch_size = 200
epoch = 60
temperature = 0.5

In [7]:
def read_data(file_name):
    '''
     open and read text file
    '''
    text = open(file_name, 'r').read()
    return text.lower()

In [8]:
def featurize(text):
    '''
     featurize the text to train and target dataset
    '''
    unique_chars = list(set(text))
    len_unique_chars = len(unique_chars)

    input_chars = []
    output_char = []

    for i in range(0, len(text) - max_len, step):
        input_chars.append(text[i:i+max_len])
        output_char.append(text[i+max_len])

    train_data = np.zeros((len(input_chars), max_len, len_unique_chars))
    target_data = np.zeros((len(input_chars), len_unique_chars))

    for i , each in enumerate(input_chars):
        for j, char in enumerate(each):
            train_data[i, j, unique_chars.index(char)] = 1
        target_data[i, unique_chars.index(output_char[i])] = 1
    return train_data, target_data, unique_chars, len_unique_chars


In [9]:
def rnn(x, weight, bias, len_unique_chars):
    '''
     define rnn cell and prediction
    '''
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, len_unique_chars])
    x = tf.split(x, max_len, 0)

    cell = tf.contrib.rnn.BasicLSTMCell(num_units, forget_bias=1.0)
    outputs, states = tf.contrib.rnn.static_rnn(cell, x, dtype=tf.float32)
    prediction = tf.matmul(outputs[-1], weight) + bias
    return prediction

In [10]:
def sample(predicted):
    '''
     helper function to sample an index from a probability array
    '''
    exp_predicted = np.exp(predicted/temperature)
    predicted = exp_predicted / np.sum(exp_predicted)
    probabilities = np.random.multinomial(1, predicted, 1)
    return probabilities


In [11]:
def run(train_data, target_data, unique_chars, len_unique_chars):
    '''
     main run function
    '''
    x = tf.placeholder("float", [None, max_len, len_unique_chars])
    y = tf.placeholder("float", [None, len_unique_chars])
    weight = tf.Variable(tf.random_normal([num_units, len_unique_chars]))
    bias = tf.Variable(tf.random_normal([len_unique_chars]))

    prediction = rnn(x, weight, bias, len_unique_chars)
    softmax = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y)
    cost = tf.reduce_mean(softmax)
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

    init_op = tf.global_variables_initializer()
    saver = tf.train.Saver()    
    sess = tf.Session()
    sess.run(init_op)

    num_batches = int(len(train_data)/batch_size)

    for i in range(epoch):
        print "----------- Epoch {0}/{1} -----------".format(i+1, epoch)
        count = 0
        for _ in range(num_batches):
            train_batch, target_batch = train_data[count:count+batch_size], target_data[count:count+batch_size]
            count += batch_size
            sess.run([optimizer] ,feed_dict={x:train_batch, y:target_batch})
        #get on of training set as seed
        seed = train_batch[:1:]
        save_path = saver.save(sess, "./tmp/rnn_model" + str(i) + ".ckpt")

        #to print the seed 40 characters
        seed_chars = ''
        for each in seed[0]:
                seed_chars += unique_chars[np.where(each == max(each))[0][0]]
        print "Seed:", seed_chars

        #predict next 1000 characters
        for i in range(1000):
            if i > 0:
                remove_fist_char = seed[:,1:,:]
                seed = np.append(remove_fist_char, np.reshape(probabilities, [1, 1, len_unique_chars]), axis=1)
            predicted = sess.run([prediction], feed_dict = {x:seed})
            predicted = np.asarray(predicted[0]).astype('float64')[0]
            probabilities = sample(predicted)
            predicted_chars = unique_chars[np.argmax(probabilities)]
            seed_chars += predicted_chars
        print 'Result:', seed_chars
    sess.close()


In [12]:
def convert_to_seed(text, unique_chars):
    len_uniques = len(unique_chars)
    letters = []
    for i in text:
        letter = np.zeros((1, len_uniques))
        for j in range(len(unique_chars)):
            if i == unique_chars[j]:
                letter[0, j] = 1.
        letters.append(letter)
    return np.array([np.vstack(letters)])
    

In [13]:
def restore_model():
    text = read_data('shakespeare.txt')
    train_data, target_data, unique_chars, len_unique_chars = featurize(text)
    x = tf.placeholder("float", [None, max_len, len_unique_chars])
    y = tf.placeholder("float", [None, len_unique_chars])
    weight = tf.Variable(tf.random_normal([num_units, len_unique_chars]))
    bias = tf.Variable(tf.random_normal([len_unique_chars]))

    prediction = rnn(x, weight, bias, len_unique_chars)
    softmax = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y)
    cost = tf.reduce_mean(softmax)
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

    init_op = tf.global_variables_initializer()
    saver = tf.train.Saver()    
    sess = tf.Session()
    sess.run(init_op)
    saver.restore(sess, "./tmp/rnn_model1.ckpt")
    
    return unique_chars, prediction, sess, x, len_unique_chars

In [14]:
def predict(text, unique_chars, prediction, sess, x, len_unique_chars):
    
    text_head = ''
    pad_head = 0
    if len(text) > 40:
        text_head = text[:-40]        
        text = text[-40:]
    elif len(text) < 40:
        pad_head  = 40 - len(text)
        text = ' ' * pad_head + text
        
    seed = convert_to_seed(text, unique_chars)
    #to print the seed 40 characters
    seed_chars = ''
    for each in seed[0]:
            seed_chars += unique_chars[np.where(each == max(each))[0][0]]
    print "Seed:", seed_chars

    
    #predict next 1000 characters
    for i in range(50):
        if i > 0:
            remove_fist_char = seed[:,1:,:]
            seed = np.append(remove_fist_char, 
                             np.reshape(probabilities, [1, 1, len_unique_chars]), axis=1)
        predicted = sess.run([prediction], feed_dict = {x:seed})
        predicted = np.asarray(predicted[0]).astype('float64')[0]
        probabilities = sample(predicted)
        predicted_chars = unique_chars[np.argmax(probabilities)]
        seed_chars += predicted_chars
        if predicted_chars == '.':
            return (text_head + seed_chars)[pad_head:]

    

In [10]:
# unique_chars, predictions, sess, x, len_unique_chars = restore_model()
# predict('out fane with you ', unique_chars, predictions, sess, x, len_unique_chars)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

INFO:tensorflow:Restoring parameters from ./tmp/rnn_model1.ckpt
Seed:                       out fane with you 


'out fane with you strake what is the bord .'

In [11]:
# text = read_data('shakespeare.txt')
# train_data, target_data, unique_chars, len_unique_chars = featurize(text)
# run(train_data, target_data, unique_chars, len_unique_chars)

In [12]:
# text = read_data('shakespeare.txt')
# train_data, target_data, unique_chars, len_unique_chars = featurize(text)
# train_batch, target_batch = train_data[1:5], target_data[1:5]
# seed = train_batch[:1:]
# print seed

In [13]:
# np.zeros((1, 5))

In [14]:
# def convert_to_seed(text, unique_chars):
#     len_uniques = len(unique_chars)
#     letters = []
#     for i in text:
#         letter = np.zeros((1, len_uniques))
#         for j in range(len(unique_chars)):
#             if i == unique_chars[j]:
#                 letter[0, j] = 1.
#         letters.append(letter)
#     return np.array([np.vstack(letters)])
    

In [15]:
# convert_to_seed('hello', unique_chars).shape

In [15]:
text = read_data('shakespeare.txt')
train_data, target_data, unique_chars, len_unique_chars = featurize(text)
run(train_data, target_data, unique_chars, len_unique_chars)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

----------- Epoch 1/60 -----------
Seed: ve him some relief if it be but for that
Result: ve him some relief if it be but for that is be this our con lort .
and so make me mand and of is our fell be i man thee vair ?
o the fare , this bat that is a dos a a the for i not to grand tree and to bear heres , but the far , my i well be mist .
chare it ake the more .
but sire a not but han , is is i have to mer to the will far hem as to be wath stres .
be come th s forge .
the i atere the sure and and and not and the sare the 'st the wist is and hear of ther is not make .
the kust the buch a dould hive .
with see that wast , is able ive it shat be dist me the cane the mare !
my lork , shourd not and withers gon and , tis a dous of the goring to mint .
and som not , gon the more the ast i mane sit lit this s

Seed: ve him some relief if it be but for that
Result: ve him some relief if it be but for that i stand wish a read the serve it .
which is me not in't a ling ready .
ay , well me stand .
what , he is stay , and they will strong thee , i go , my brother , when i have strong you a good not , and this be not make sinch and make the clove .
there is now him rish more to the heart with mark men and mine strange , and strigh .
what , is a man in the safe from that in her done in the other destre mossed .
the wind i do be for your hand .
what is me stand .
i will be die foul a sold .
let me me .
there's were a dife and a present .
you were inserve the sweet but she disp and the cibon .
there's some stoon and strange the wordd , and the mitter of the streat , and dream .
we have seem the carring for the wind ready , and i am the dumble and shall be dear , i pray he mistress .
what is the bring on the slan .
a pray thee say the gentle sold on me but , which is a little .
i am the duman .
there

Seed: ve him some relief if it be but for that
Result: ve him some relief if it be but for that heard in the mistress and the bast of the grace bear , i warrant you , king of this father .
have i speak to such distreason for you shall be strange .
let him this great and the strengt the mighty shall be sail .
the world heard .
with a summand and make the world .
there be some think they sail , and thou ussest with luce and so in the insure they were i shall not him dromina with the more that i be son and sound ?
what say is a king .
what do not see , that i may with a heard , and i will some men .
i were thee like a dill  matter , so .
we have not a woman would end her fall .
and there will indeed him and the mist a bed them .
what is the worthing .
oh , what is he is a villain , and the world i should be with me .
this is the world .
i pray you , sir .
i have can not will me serve the bound of the wind , and strange me the little part , and the hour married man , sir , be mine on me .


Seed: ve him some relief if it be but for that
Result: ve him some relief if it be but for that i mean but the married .
the time is not give thee .
what's the march being wear the dream not a book .
i think i can not the strick of the strick better .
therefore , good sir , that i should be with mine eyes of all the mist with .
i will hear the brother .
i pray you , wind that i long is the prince , sir , for you do beg , that mind make a revolt of the more to be not .
and i am a gentleman that i will dream her from her heart , and he is the more that i mistress sea , to the every daughter .
and fortune , so ?
i think the world wish their hand .
heavens , who , that is the prince ?
i am the sing the married me it and the sold of mine to the read instant .
there is no heard .
marry , the more i have done the ward and the more that he is unsud the more the way .
what is the good sense have speed and bear .
i shall we do to be not ball , and the sudge and place , the day .
we have been a f

Seed: ve him some relief if it be but for that
Result: ve him some relief if it be but for that she was me and heart a trible , to have no more .
and we shall be some print shall be friend we with her .
he will do not his saw a charinge ?
what is your mistress as they were blost .
i have a man , the horse with the pite of her babes and be deed .
i am a company .
this is gone ?
i pray you , sir , and the will us all of a lian .
the time of the strength see their sing were sold .
how now , peace , if my sea , how hath the about a daughter .
hear me , capen interrown .
the duke of proson , my lord , and there be shords , be the lady rankate .
the streads will make it .
the companions and soldier that i have seek the more were be the duke here , being see wenches me .
the string in such shop dost thou shalt be done .
there is no fair braggent like a doth .
i were not a soldier way .
then i am a command .
i'll marry what is me , and will this reason , a heart .
i am a dirult for his present

Seed: ve him some relief if it be but for that
Result: ve him some relief if it be but for that would bear the night , for a back were but the wind's fat love this dream .
and and so to be much to the order of this a name of a knave in his and but she hath see thee .
make and speak to him that i say .
he says the prince were is , i am bady .
we will hear gone .
sir , i do not make a liberty .
we will be .
i will go to be the more which i shall be so .
what are the sack and said to the sing .
what is the day .
heaven offer dead , and then she is the duke and the sleepor of all rest .
the hoping of the world , and will so repose , i pray thee , that you do appear it and brought of thee .
there is no fair braggmen .
i will see them the brother .
but in the mind , thou hast safe weak a speed .
these dead in my son often , and there is a house .
a some them as death .
and there make a maid the brother spirit .
i am the work of a so fire , sir .
i pray you , do not bring thee .
there's he sa

Seed: ve him some relief if it be but for that
Result: ve him some relief if it be but for that we will use me .
a soldier from me strange , and not a fine of mine of the thing fair weary .
the strange for her curse of thee plain , sir , i have been a devils , i am sorry .
what art thou lost not from my sings of good curse is a prepentan .
we'll , thou met it .
what , that's more from the devil shall not be not .
there is to the bed be saw with sadam .
what would they come , they shall be the parting back .
i have no more than served him a time to do thee fall the house is now they with his bring me as a some money .
and there is the banded wit of the strange , and in the sleep well repent .
fairy knave , or is this business .
we will report all the wind's gone , beat me to the seen , and they have doth make the chark a traible in the royal on the begate with your friends , and their brother to me .
what would it stand it .
i do not like a my rest from the sleep .
i should see thee wel

Seed: ve him some relief if it be but for that
Result: ve him some relief if it be but for that i have and come to my wife .
sir , i should be not , and good compase .
what spirits she that come then , and we sling the mind ranks .
is it pack all the then they were self a devil , since i thank you .
and there , did head it is this the sure .
what spoke thee for me .
i will see you , when i have i will speak all this as of the more .
thou hast said his pains that he was the day .
sweet married work ?
there , not made , when i do this prove more demains the strange well tranio , i will not confess , that i have the mine , send to my sleep , and there she is well .
what are thou think you do best thou are , let me , a my come to this well .
what say you , thou lord's death .
and i will be served me to be so .
what a thousand thou art more soldier , though all this name it hath more than the heavens high and me to every spoke a command .
what may as he is .
thou didst then do the world in 